In [4]:
import os
import json
import pandas as pd

folder_path = '/home/jupyter-23521027/refresh-bert/data/liputan6_data/canonical/train' 

file_list = os.listdir(folder_path)
json_data_list = []
file_list = os.listdir(folder_path)

for file_name in file_list:
    file_path = os.path.join(folder_path, file_name) 

    if os.path.isfile(file_path):
        with open(file_path, 'r') as file:
            json_data = json.load(file)
            json_data_list.append(json_data)

df = pd.DataFrame(json_data_list)

In [5]:
df

,id,url,clean_article,clean_summary,extractive_summary
0,48951,https://www.liputan6.com/news/read/48951/polis...,"[[Liputan6, ., com, ,, Lamongan, :, Tim, Inves...","[[Senjata, api, dan, ratusan, butir, peluru, m...","[1, 2]"
1,124973,https://www.liputan6.com/news/read/124973/gent...,"[[Liputan6, ., com, ,, Depok, :, Abdul, Madjid...","[[Genta, buatan, Abdul, Madjid, bisa, melahirk...","[0, 1]"
2,298677,https://www.liputan6.com/news/read/298677/wasp...,"[[Liputan6, ., com, ,, Bekasi, :, Aparat, Kepo...","[[Berhati-hatilah, Anda, bila, bepergian, ke, ...","[0, 5]"
3,40569,https://www.liputan6.com/news/read/40569/imbau...,"[[Liputan6, ., com, ,, Denpasar, :, Imbauan, M...","[[Peringatan, untuk, tidak, pergi, ke, Indones...","[5, 8]"
4,242869,https://www.liputan6.com/news/read/242869/rapa...,"[[Liputan6, ., com, ,, Jakarta, :, Gempa, kuat...","[[Rapat, pleno, KPU, terhenti, ketika, gempa, ...","[0, 4]"
...,...,...,...,...,...
193878,164143,https://www.liputan6.com/news/read/164143/tahu...,"[[Liputan6, ., com, ,, Jakarta, :, Warga, yang...","[[Dirjen, Pajak, membuat, aturan, setiap, oran...","[2, 4]"
193879,150247,https://www.liputan6.com/news/read/150247/mass...,"[[Liputan6, ., com, ,, Bogor, :, Ribuan, umat,...","[[Ribuan, umat, Islam, menyerbu, vila, milik, ...","[0, 1]"
193880,204203,https://www.liputan6.com/news/read/204203/bate...,"[[Chelsea, melayangkan, keberatannya, kepada, ...","[[Ketua, Leeds, United, ,, Ken, Bates, berhasi...","[0, 13]"
193881,159188,https://www.liputan6.com/news/read/159188/peny...,"[[Liputan6, ., com, ,, Denpasar, :, Kebakaran,...","[[Kebakaran, toko, sepatu, di, Denpasar, ,, Ba...","[0, 3]"


In [6]:
df['sentences'] = df['clean_article'].apply(lambda x: [' '.join(sentence) for sentence in x])

In [7]:
df.head(1)

,id,url,clean_article,clean_summary,extractive_summary,sentences
0,48951,https://www.liputan6.com/news/read/48951/polis...,"[[Liputan6, ., com, ,, Lamongan, :, Tim, Inves...","[[Senjata, api, dan, ratusan, butir, peluru, m...","[1, 2]","[Liputan6 . com , Lamongan : Tim Investigasi B..."


In [13]:
import nltk
from nltk.tokenize import word_tokenize
from datetime import datetime

# Preprocessing step: tokenization and cleaning

# Define the context window size
window_size = 2

# Open the text file
with open('/home/jupyter-23521027/refresh-bert/data/indo4b/dataset_all_uncased_blankline.txt', 'r') as file:
    # Iterate over each line in the file
    now = datetime.now()
    word_pairs = []

    for l, line in enumerate(file):
        # Tokenize the line into words
        words = word_tokenize(line)
        
        # Generate word pairs
        for i, target_word in enumerate(words):
            for j in range(i - window_size, i + window_size + 1):
                if j != i and j >= 0 and j < len(words):
                    context_word = words[j]
                    word_pairs.append((target_word, context_word))
                    
        if ( l + 1 ) % 100000 == 0:
            print(f"processed {l+1} files in : {datetime.now() - now}")
            now = datetime.now()
        
        if l == 500000:
            break

# Assuming you have word_pairs as a list of word pairs

# Create word-to-index and index-to-word mappings
words = set([pair[0] for pair in word_pairs] + [pair[1] for pair in word_pairs])
word_to_index = {word: index for index, word in enumerate(words)}
index_to_word = {index: word for index, word in enumerate(words)}

# Convert word pairs to numerical representations
numeric_pairs = [(word_to_index[pair[0]], word_to_index[pair[1]]) for pair in word_pairs]

processed 100000 files in : 0:00:12.228631
processed 200000 files in : 0:00:11.853490
processed 300000 files in : 0:00:11.881087
processed 400000 files in : 0:00:11.848886
processed 500000 files in : 0:00:11.568216


In [ ]:
from datetime import datetime

MAX_DOC_LENGTH = 100
DATASET_SIZE = 5000
docs = []

now = datetime.now()
file = open("/home/jupyter-23521027/refresh-bert/data/preprocessed-input-directory/liputan6.training.doc" , 'w')

for i, row in df.iterrows():
    file.write(f'liputan6-{str(row["id"])}' + "\n")

    sentences = list(row["sentences"][:MAX_DOC_LENGTH])
    for sentence in sentences:
        for word in sentence:
            try:
                index = str(word_to_index[word])
            except Exception as KeyError:
                index = "-1"
            file.write(index + " ")
            
        file.write("\n")

    if (i + 1) % 100 == 0:
        print(f"processing {i + 1} with time: {datetime.now() - now}")
        now = datetime.now()

#     if (i + 1) == DATASET_SIZE:
#         break

    file.write("\n")
    
file.close()

In [18]:
!cat /home/jupyter-23521027/refresh-bert/data/preprocessed-input-directory/liputan6.training.doc | head -c 10000

liputan6-48951
-1 164897 49805 105706 105613 2002 71427 61886 -1 128812 -1 57709 68173 162683 -1 12100 -1 -1 2002 162683 68173 71427 163473 2002 71427 -1 79949 -1 -1 164897 162683 -1 -1 71427 51467 161519 131123 105613 164897 163473 2002 131123 164897 -1 -1 68173 162683 -1 -1 2002 100316 164897 -1 133552 2002 49556 164897 -1 -1 161519 49805 68173 100316 164897 131123 164897 2002 71427 -1 -1 161519 131123 68173 49556 -1 -1 2002 162683 68173 71427 163473 2002 71427 -1 12100 -1 -1 42031 2002 133552 -1 42561 -1 66524 135525 66524 -1 152732 -1 12100 -1 162683 161519 71427 161519 162683 105706 26779 2002 71427 -1 100316 2002 163473 164897 -1 131123 161519 49805 105706 57709 105706 26779 -1 131123 161519 71427 7858 2002 105613 2002 -1 2002 49805 164897 -1 7858 161519 71427 164897 131123 -1 -1 -1 131319 159713 61886 -1 133552 2002 71427 -1 118835 66524 151746 -1 26861 105706 105613 164897 49556 -1 49805 161519 100316 105706 49556 105706 -1 133552 2002 49556 164897 -1 26861 161519 49556 26861 2